In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings

In [3]:
warnings.filterwarnings('ignore')

In [4]:
os.chdir('..')

In [17]:
import requests
from Bio import SeqIO
from mofiwo.connector.ensembl import read_ensembl

In [6]:
ret_esm = read_ensembl('ENSMUST00000037811', req_param={'type':'cds'})

In [7]:
from mofiwo.utility.rna_handler import load_rna_fasta_zipfile

In [8]:
fasta_cds_zipfile = r'D:\workspace\rnamotif\samples\s1_cds.zip'
fasta_cdna_zipfile = r'D:\workspace\rnamotif\samples\s1_cdna.zip'
#fasta_zipfile = r'C:/Users/byeun/Downloads/s1_cds.zip'

In [9]:
dic_cds = load_rna_fasta_zipfile(fasta_cds_zipfile)
dic_cdna = load_rna_fasta_zipfile(fasta_cdna_zipfile)

In [10]:
from mofiwo.utility.rna_handler import generate_3utr_from_cdna_cds

In [ ]:
dic_utr3 = generate_3utr_from_cdna_cds(dic_cdna, dic_cds)

In [16]:
foo = dic_utr3[next(iter(dic_utr3))]

In [20]:
SeqIO.write(foo, r'../fasta_file_example.fasta', 'fasta')

1

In [21]:
from Bio.motifs import meme

In [22]:
with open(r'C:\Users\by003457\Downloads\meme.xml') as fin:
    records = meme.read(fin)

In [23]:
motifs = [x for x in records]

In [26]:
motif = motifs[0]

In [29]:
motif.name

'WWKVTTTTTWTKTTTKTTTKTKTKTKTKTTTDTWTNTDTKW'

In [35]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options

In [36]:
opts = Options()
opts.set_headless()

In [39]:
browser = Chrome(r'D:\tools\chromedriver_win32_v88\chromedriver.exe',options=opts)

In [40]:
browser.get('https://duckdukgo.com')

WebDriverException: Message: unknown error: net::ERR_SSL_PROTOCOL_ERROR
  (Session info: headless chrome=88.0.4324.190)
